# Convolutional Neural Network with Pytorch

### Model A: 
<li> 2 convoltional Layers (Same padding (same output size))
<li> 2 Max pooling layers
<li> 1 Fully Connected Layers

#### Steps: 

<li> Step 1: Load Dataset
<li> Step 2: Make Dataset Iterable
<li> Step 3: Create Model Class
<li> Step 4: Instantiate Model Class
<li> Step 5: Instantiate Loss Class
<li> Step 6: Instantiate Optimizer Class
<li> Step 7: Train Model

###### Step 1: Load Dataset


In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable


In [2]:
train_dataset = dsets.MNIST(root = './data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)

test_dataset = dsets.MNIST(root = './data',
                           train = False,
                           transform = transforms.ToTensor())

In [4]:
print(train_dataset.data.size())

torch.Size([60000, 28, 28])


In [8]:
print(train_dataset.targets.size())

torch.Size([60000])


In [9]:
print(test_dataset.data.size())

torch.Size([10000, 28, 28])


In [10]:
print(test_dataset.targets.size())

torch.Size([10000])


###### Step 2: Make Dataset Iterable

In [11]:
batch_size = 100
n_iters = 3000

num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)

###### Step 3: Create Model Class

In [14]:
class CNNModel(nn.Module):
    
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional Model 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Maxpool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolutional Model 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Maxpool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fn1 = nn.Linear(32 * 7 * 7, 10)
        
        
        
    def forward(self, x):
        
        out = self.cnn1(x)
        out = self.relu1(out)
        
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.maxpool2(out)
        
        out = out.view(out.size(0), -1)

        out = self.fn1(out)
        
        return out

###### Step 4: Instantiate Model Class

In [15]:
model = CNNModel()

###### Step 5: Instantiate Loss Class

In [16]:
criterion = nn.CrossEntropyLoss()

###### Step 6: Instantiate Optimizer Class

In [17]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**Parameters In-Depth**



In [18]:
print(model.parameters())

print(len(list(model.parameters())))

print(list(model.parameters())[0].size())

print(list(model.parameters())[1].size())

print(list(model.parameters())[2].size())

print(list(model.parameters())[3].size())

print(list(model.parameters())[4].size())

print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x000001DC0591B728>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


###### Step 7: Train Model

Process

1. <b>Convert input/labels to variables</b>
    * CNN input: (1, 28, 28)
    * Feedforward NN input: ( 1, 28*28)
2. Clear gradient buffers
3. Get output given inputs
4. Get loss
5. Get gradients w.r.t. parameters
6. Update parameters using gradients
    * parameters = parameters - leaarning_rate * parameter_gradients
7. Repeat

In [21]:
iter = 0


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                
                images = Variable(images)
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct / total
            
            print('Iterations: {}, Loss: {}, Accuracy: {}'.format(iter, loss.data, accuracy.float()))

Iterations: 500, Loss: 0.18022538721561432, Accuracy: 94.0
Iterations: 1000, Loss: 0.18132656812667847, Accuracy: 95.0
Iterations: 1500, Loss: 0.13275887072086334, Accuracy: 96.0
Iterations: 2000, Loss: 0.17142516374588013, Accuracy: 96.0
Iterations: 2500, Loss: 0.09967418760061264, Accuracy: 97.0
Iterations: 3000, Loss: 0.05589468404650688, Accuracy: 97.0


### Model B: 
<li> 2 convoltional Layers (Same padding (same output size))
<li><b> 2 Average pooling layers</b>
<li> 1 Fully Connected Layers

#### Steps: 

<li> Step 1: Load Dataset
<li> Step 2: Make Dataset Iterable
<li> Step 3: Create Model Class
<li> Step 4: Instantiate Model Class
<li> Step 5: Instantiate Loss Class
<li> Step 6: Instantiate Optimizer Class
<li> Step 7: Train Model

In [22]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOAD DATASET
'''
train_dataset = dsets.MNIST(root = './data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)

test_dataset = dsets.MNIST(root = './data',
                           train = False,
                           transform = transforms.ToTensor())
'''
STEP 2: MAKE DATASET ITERABLE
'''
batch_size = 100
n_iters = 3000

num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)
'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional Model 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Average pool 1
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        
        # Convolutional Model 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Average pool 2
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fn1 = nn.Linear(32 * 7 * 7, 10)
        
        
        
    def forward(self, x):
        
        out = self.cnn1(x)
        out = self.relu1(out)
        
        out = self.avgpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.avgpool2(out)
        
        out = out.view(out.size(0), -1)

        out = self.fn1(out)
        
        return out
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
model = CNNModel()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                
                images = Variable(images)
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct / total
            
            print('Iterations: {}, Loss: {}, Accuracy: {}'.format(iter, loss.data, accuracy))

Iterations: 500, Loss: 0.6559540629386902, Accuracy: 81
Iterations: 1000, Loss: 0.5728594064712524, Accuracy: 87
Iterations: 1500, Loss: 0.33369237184524536, Accuracy: 89
Iterations: 2000, Loss: 0.4286341071128845, Accuracy: 91
Iterations: 2500, Loss: 0.32874199748039246, Accuracy: 92
Iterations: 3000, Loss: 0.23829668760299683, Accuracy: 92


In [30]:
import gc
gc.collect()

687

### Average Pooling Test Accuracy < Max Pooling Test Accuracy

### Model C: 
<li> 2 convoltional Layers (Valid padding (smaller output size))
<li><b> 2 Max pooling layers</b>
<li> 1 Fully Connected Layers

#### Steps: 

<li> Step 1: Load Dataset
<li> Step 2: Make Dataset Iterable
<li> Step 3: Create Model Class
<li> Step 4: Instantiate Model Class
<li> Step 5: Instantiate Loss Class
<li> Step 6: Instantiate Optimizer Class
<li> Step 7: Train Model

In [28]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOAD DATASET
'''
train_dataset = dsets.MNIST(root = './data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)

test_dataset = dsets.MNIST(root = './data',
                           train = False,
                           transform = transforms.ToTensor())
'''
STEP 2: MAKE DATASET ITERABLE
'''
batch_size = 100
n_iters = 3000

num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)
'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional Model 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Average pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolutional Model 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Average pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fn1 = nn.Linear(32 * 4 * 4, 10)
        
        
        
    def forward(self, x):
        
        out = self.cnn1(x)
        out = self.relu1(out)
        
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.maxpool2(out)
        
        out = out.view(out.size(0), -1)

        out = self.fn1(out)
        
        return out
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
model = CNNModel()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                
                images = Variable(images)
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct / total
            
            print('Iterations: {}, Loss: {}, Accuracy: {}'.format(iter, loss.data, accuracy))

Iterations: 500, Loss: 0.561103105545044, Accuracy: 87
Iterations: 1000, Loss: 0.3597143888473511, Accuracy: 93
Iterations: 1500, Loss: 0.17969940602779388, Accuracy: 94
Iterations: 2000, Loss: 0.24145840108394623, Accuracy: 95
Iterations: 2500, Loss: 0.1256866604089737, Accuracy: 96
Iterations: 3000, Loss: 0.13181596994400024, Accuracy: 96


### Summary of Results

|     Model A      |     Model B        |   Model C         |
|------------------|--------------------|-------------------|
|   Max Pooling    |  Average Pooling   |  Max Pooling      |
|   Same Padding   |  Same Padding      |  Valid Padding    |
|      > 97 %      |      92 %          |        96 %       |

## Convolutional Neural Network with Pytorch (GPU)

#### Steps: 

<li> Step 1: Load Dataset
<li> Step 2: Make Dataset Iterable
<li> Step 3: Create Model Class
<li> Step 4: Instantiate Model Class
<li> Step 5: Instantiate Loss Class
<li> Step 6: Instantiate Optimizer Class
<li> Step 7: Train Model

In [31]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOAD DATASET
'''
train_dataset = dsets.MNIST(root = './data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)

test_dataset = dsets.MNIST(root = './data',
                           train = False,
                           transform = transforms.ToTensor())
'''
STEP 2: MAKE DATASET ITERABLE
'''
batch_size = 100
n_iters = 3000

num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)
'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional Model 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Average pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolutional Model 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Average pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fn1 = nn.Linear(32 * 7 * 7, 10)
        
        
        
    def forward(self, x):
        
        out = self.cnn1(x)
        out = self.relu1(out)
        
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.maxpool2(out)
        
        out = out.view(out.size(0), -1)

        out = self.fn1(out)
        
        return out
    
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
model = CNNModel()

if torch.cuda.is_available():
    model.cuda()

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else: 
            images = Variable(images)
            labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                if torch.cuda.is_available():
                    images = Variable(images.cuda())

                else: 
                    images = Variable(images)
                    
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else: 
                    correct += (predicted == labels).sum()
                    
                    
            accuracy = 100 * correct / total
            
            print('Iterations: {}, Loss: {}, Accuracy: {}'.format(iter, loss.data, accuracy))

Iterations: 500, Loss: 0.3509790003299713, Accuracy: 90
Iterations: 1000, Loss: 0.2690621614456177, Accuracy: 92
Iterations: 1500, Loss: 0.3044084906578064, Accuracy: 94
Iterations: 2000, Loss: 0.17182093858718872, Accuracy: 95
Iterations: 2500, Loss: 0.07065372914075851, Accuracy: 96
Iterations: 3000, Loss: 0.11300769448280334, Accuracy: 96


-------------